In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./part.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_part_number = df['Part_Number'].tolist()
list_brand = df['Brand'].tolist()
list_url = ['https://ecatalog.smpcorp.com/V2/STD/api/image/getallimages?zoomFactor_sm=75&zoomFactor_md=360&zoomFactor_bg=960&partNum=' + part_number + '&brand=' + list_brand for part_number, list_brand in zip(list_part_number, list_brand)]

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests
import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for part_number, url in zip(list_part_number, list_url):
    work.put_nowait((part_number, url))

def crawler():
    global df
    
    while not work.empty():
        part_number, url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(101):
            try:
                resp = json.loads(requests.get(url,
                                               headers=UA.get_User_Agent_Requests(),
                                               proxies=Proxy.get_Proxy_Requests()).text)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Part_Number': part_number,
                                         'Src': resp[0]['image_BG_Url']}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Part_Number': part_number}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(part_number +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(5):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./src_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：498

APK103  <->  [ok] - 剩余数量：493
APK100  <->  [ok] - 剩余数量：492
APS101  <->  [ok] - 剩余数量：491
APK102  <->  [ok] - 剩余数量：490
APS103  <->  [ok] - 剩余数量：489
APS102  <->  [ok] - 剩余数量：488
APS104  <->  [ok] - 剩余数量：487
APS105  <->  [ok] - 剩余数量：486
APS108  <->  [ok] - 剩余数量：485
APS106  <->  [ok] - 剩余数量：484
APS107  <->  [ok] - 剩余数量：483
APS109  <->  [ok] - 剩余数量：482
APS111  <->  [ok] - 剩余数量：481
APS112  <->  [ok] - 剩余数量：480
APS114  <->  [ok] - 剩余数量：479
APK101  <->  [ok] - 剩余数量：478
APS116  <->  [ok] - 剩余数量：477
APS115  <->  [ok] - 剩余数量：476
APS120  <->  [ok] - 剩余数量：475
APS117  <->  [ok] - 剩余数量：474
APS125  <->  [ok] - 剩余数量：473
APS123  <->  [ok] - 剩余数量：472
APS126  <->  [ok] - 剩余数量：471
APS118  <->  [ok] - 剩余数量：470
APS127  <->  [ok] - 剩余数量：469
APS128  <->  [ok] - 剩余数量：468
APS131  <->  [ok] - 剩余数量：467
APS133  <->  [ok] - 剩余数量：466
APS134  <->  [ok] - 剩余数量：465
APS135  <->  [ok] - 剩余数量：464
APS132  <->  [ok] - 剩余数量：463
APS137  <->  [ok] - 剩余数量：462
APS136  <->  [ok] - 剩余数量：461
APS138  <->  [ok] - 剩余数量：460
APS13

APS458  <->  [ok] - 剩余数量：210
APS467  <->  [ok] - 剩余数量：209
APS468  <->  [ok] - 剩余数量：208
APS469  <->  [ok] - 剩余数量：207
APS472  <->  [ok] - 剩余数量：206
APS473  <->  [ok] - 剩余数量：205
APS470  <->  [ok] - 剩余数量：204
APS474  <->  [ok] - 剩余数量：203
APS477  <->  [ok] - 剩余数量：202
APS475  <->  [ok] - 剩余数量：201
APS478  <->  [ok] - 剩余数量：200
APS479  <->  [ok] - 剩余数量：199
APS140  <->  [error] - 剩余数量：198
APS481  <->  [ok] - 剩余数量：197
APS483  <->  [ok] - 剩余数量：196
APS480  <->  [ok] - 剩余数量：195
APS485  <->  [ok] - 剩余数量：194
APS486  <->  [ok] - 剩余数量：193
APS484  <->  [ok] - 剩余数量：192
APS490  <->  [ok] - 剩余数量：191
APS488  <->  [ok] - 剩余数量：190
APS471  <->  [ok] - 剩余数量：189
APS491  <->  [ok] - 剩余数量：188
APS492  <->  [ok] - 剩余数量：187
APS496  <->  [ok] - 剩余数量：186
APS497  <->  [ok] - 剩余数量：185
APS494  <->  [ok] - 剩余数量：184
APS493  <->  [ok] - 剩余数量：183
APS498  <->  [ok] - 剩余数量：182
APS503  <->  [ok] - 剩余数量：181
APS499  <->  [ok] - 剩余数量：180
APS476  <->  [ok] - 剩余数量：179
APS505  <->  [ok] - 剩余数量：178
APS502  <->  [ok] - 剩余数量：177
APS504  <->